# Sentiment Analysis

* 文本的正負向判斷，機器學習，情感分析。
* NaiveBayesClassifier 分類器的應用。

# machine learning 機器學習
* 人們透過「經驗」來學習，機器透過「數據」來學習。
* 「監督式學習」：訓練組( training data)，測試組 (test data), 目標文本(target data)。
* 針對訓練組( training data) 訓練，找出一些分類的規則，接著再針對訓練組(test data)，看看依據這些分類規則進行分類，準確度有多高。如果滿意了，再針對目標文本(target data)進行分類預測。
* 學習的演算法有多種，通常要利用類神經網絡來進行，蒐集的文件要足夠多，且恰當，分類的正確率即能提高。

# training set


In [ ]:
path='d:/My python/'

In [ ]:
# 正向的文本，含標記，請注意格式, list of tuple, tuple 內 第一個元素為 文本，第二個為 標記
pos_tweets = [('我 喜歡 車子', 'positive'),
              ('我 太 高興','positive'),
              ('他 太棒 了', 'positive'),
              ('我 覺得 心情好', 'positive'),
              ('音樂會 令人 感動', 'positive'),
              ('我 的 這個 好 朋友。','positive')] 

In [ ]:
# 負向的文本，含標記，請注意格式, list of tuple, tuple 內 第一個元素為 文本，第二個為 標記
neg_tweets = [('我 討厭 搭 公車', 'negative'),
              ('他 太 可怕 了', 'negative'),
              ('我 覺得 很 累', 'negative'),
              ('我 不想 參加 這個 研討會', 'negative'),
              ('我 不喜歡', 'negative'),
              ('早上 是 競爭 對手', 'negative')]

In [ ]:
posts=pos_tweets+neg_tweets

In [ ]:
# ngrams 函數，以多個字元作為分析單位
def ngrams(input, min, max):
    input = input.split(' ')
    output = []
    texts=''  
    for c in range(min,max+1):
        n=c
        for i in range(len(input)-n+1):
             texts=' '.join(input[i:i+n])
             output.append(texts)
    return output

## 化為文字袋，標記正向或反向。

In [ ]:
train_set = []
# 把測試組的所有資料叫進來，含字詞及情緒標記
for (words, sentiment) in posts:
    # 所有的字詞 與 情緒標記 ngram, min, max ##################
    word_list=ngrams(words, 1,1) 
    
    # 去除停用字 stopwords
    #stopwords=['我', '很', '他', '早上', '的', '這個', '也', '一點']    
    #word_list=[e for e  in word_list if e not in stopwords] 
    
    # 字長控制
    words_filtered = [e for e in word_list if len(e) >= 1]        
    train_set.append((words_filtered, sentiment))


In [ ]:
# 文字袋標準格式，list of words, 含標記, tuple
train_set

# extract features of training set

In [ ]:
import nltk
# 取得文本中所有的字詞 list，有重覆
def get_words_in_tweets(train_set):
    all_words = []
    for (words, sentiment) in train_set:
        all_words.extend(words)
    return all_words
# 取得字詞列表，dictionary_key 不重復 
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features
# 字詞是否(True or False)在文件有被引用，dictionary
def extract_features(train_set):
    document_words = set(train_set)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

## 單一文本featurs與traning set 比較

In [ ]:
# 字詞列表，不重復 dictionary_key
get_words = get_words_in_tweets(train_set)
get_words

In [ ]:
# 字詞列表， dictionary_key 不重復
word_features = get_word_features(get_words_in_tweets(train_set))
word_features

## 文字袋比對，是否有某個字詞。

In [ ]:
# 文字袋比對，是否有某個字詞。注意引數 extract_features, tweets　
training = nltk.classify.apply_features(extract_features, train_set)

# 訓練用資料共有十筆，依序顯示　training_set 內容
for n in training:
    print (n)
    print ('-----------------------------------------')

## NaiveBayesClassifier 分類器，各關鍵字出現與否，類別的機率

In [ ]:
# 找出潛在的規則
classifier = nltk.NaiveBayesClassifier.train(training)
# 列出最重要的 50 條規則
print (classifier.show_most_informative_features(500))
# 第一條規則：如果沒有 好 的話，負向的機率會是正向的 1.4 倍

## 儲存 classifier

In [ ]:
import pickle
save_classifier = open(path+"data/naivebayes_test.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

## 呼叫 classifier

In [ ]:
import pickle
classifier_f = open(path+"data/naivebayes_test.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

* 簡單測試

In [ ]:
test = '我 很 累'
print (classifier.classify(extract_features(test.split())))


# Test set

In [ ]:
test_tweets =[('張三 是 我 的 壞 朋友','negative'),
              ('我 不 喜歡 那個人','negative'),
              ('那 首 歌 很 棒','positive'),
              ('那 房子 令人 討厭','negative')]

## 針對 test set 進行預測

In [ ]:
print ('predict', 'real')
print ('---------------')
# 一筆筆逐一預測
for t in test_tweets:
    pred= classifier.classify(extract_features(t[0].split()))
    print (pred, t[1])

## 計算預測的正確率

In [ ]:
yes=0
n=0
for t in test_tweets:
    pred= classifier.classify(extract_features(t[0].split()))
    n+=1
    if pred == t[1]:
        yes+=1
acc=yes/n
print (acc)

# Target Set

In [ ]:
# 目標測試，沒有情感標記，用我們的模型預測
target_tweets =['他 的 歌 吵死 人 了',
              '我 不 知道',  
              '他 過去 紀錄 不 甚 好']
for t in target_tweets:
    print (classifier.classify(extract_features(t.split())))

* 準確率不高，因為我們學習的資料不足。

In [ ]:
# 2

# 多類別的預測

* 多標籤分類問題而言，一個樣本可能同時屬於多個類別。如一個新聞屬於多個話題。這種情況下，因變數 y 需要使用一個矩陣表達出來。
* 多類別分類指的是 y 的可能取值大於2，但是 y 所屬類別是唯一的。

In [ ]:
pos_tweets = [('我 喜歡 車子', 'A'),
              ('早上 我 很 高興','A'),
              ('他 真是 太棒 了', 'A'),
              ('早上 我 覺得 心情 很 好', 'A'),
              ('昨天 晚上 的 音樂會 令人 感動', 'A'),
              ('他 是 我 最 要 好 的 朋友', 'A')]
neg_tweets = [('我 討厭 搭 公車', 'B'),
              ('這個 經歷 實在 太 可怕 了', 'B'),
              ('早上 我 覺得 很 累', 'B'),
              ('我 一點 也 不 期待 參加 這個 研討會', 'B'),
              ('我 不喜歡 他', 'B'),
              ('他 是 我 的 競爭 對手', 'B')]
no_tweets =[('不 好 也 不 壞','C'),
              ('實在 是 很 普通 啦','C'),
              ('沒有 什麼 特別 的 感覺','C'),
              ('我 不 知道','C')]

In [ ]:
import nltk
def get_test_group(total_test):
    tweets = []
    # 把測試組的所有資料叫進來，含字詞及情緒標記
    for (words, sentiment) in total_test:
        # 所有的字詞 與 情緒標記
        word_list=ngrams(words, 1,1) 
        # 去除停用字 stopwords
        stopwords=['我', '很', '他', '的', '也' ]   
        word_list=[e for e  in word_list if e not in stopwords] 

        # 字長控制
        words_filtered = [e for e in word_list if len(e) >= 1]        
        tweets.append((words_filtered, sentiment))
    return tweets   

# 文本中所有的字詞 list
def get_words_in_train(train_set):
    all_words = []
    for (words, sentiment) in train_set:
        all_words.extend(words)
    return all_words
# 字詞列表，不重復 dictionary_key
def get_word_features(all_words):
    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    return word_features
# 字詞是否在文件有被引用，dictionary
def extract_features(train_set):
    document_words = set(train_set)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [ ]:
total_test=pos_tweets + neg_tweets + no_tweets
train_set=get_test_group(total_test)
word_features = get_word_features(get_words_in_train(train_set))

# 文字袋比對，注意引數 extract_features, tweets　
training = nltk.classify.apply_features(extract_features,train_set)

# 找出潛在的規則 classifier
classifier = nltk.NaiveBayesClassifier.train(training)

# 列出最重要的 50 條規則
print (classifier.show_most_informative_features(50))


In [ ]:
Test_tweets =[('張三 很 普普通通 啦','C'),
              ('我 不喜歡 那個人','B'),
              ('那 首 歌 太 吵 了','B'),
              ('那 房子 令人 討厭','B')]

In [ ]:
yes=0
n=0
print ('predict', 'real')
print ('---------------')
for t in Test_tweets:
    pred= classifier.classify(extract_features(t[0].split()))
    pred= classifier.classify(extract_features(t[0].split()))
    print (pred, t[1])    
    n+=1
    if pred == t[1]:
        yes+=1
acc=yes/n
print ('---------------')
print (acc)


In [ ]:
target_tweets =['他 確實 是 太棒 啦',
              '他 真 的 很 討厭',  
              '沒有 什麼 特別 的 感覺']

for t in target_tweets:
    print (classifier.classify(extract_features(t.split())))

In [ ]:
# 3

--------------------------------------
# 用我們自己的資料

## 資料重整
* 讀取自建的正向語料庫與負向語料庫，檔案頗大。檔案總管檢查。
* 訓練自己的分類器。頗花時間。
* 檢測正確率
* 繁簡字體轉換：https://name.longwin.com.tw/twcn.php

In [129]:
#####################################################################
import pandas as pd
path='d:/My python/'
df=pd.read_excel(path+"corpus/情緒文本P_N.xlsx",index_col=None)
df

,text,tag
0,:怪怪~~這麼牛逼啊，身邊誰再買蒙牛，就罵誰SB！,N
1,“”加錯位置了，抵制蒙牛“很多年”,N
2,一系列的醜聞表明蒙牛的管理層素質太差，公司治理混亂。消費者應拒絕購買其產品。,N
3,一直都在抵制與其抵制日貨，不如抵制蒙牛。,N
4,又是蒙牛 再也不能吃蒙牛了，蒙牛的任何產品都不買了,N
5,已經不喝牛奶了 已經不買蒙牛了 次奧2 次奧 還有人買蒙牛？！,N
6,已經不買蒙牛好幾個月了，堅持到丫倒閉,N
7,"不愛喝蒙牛, 以後也不會喝.",N
8,"支援中國乳業,支援蒙牛!",P
9,牛奶，永不喝蒙牛！@蒙牛乳業,N


## pos_set

In [130]:
df_pos=df[df.tag=="P"]
len(df_pos)

3036

In [136]:
def 去特殊符號(t):
    import re
    # 特殊符號可以自己增補
    t = re.sub(r"[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",t)
    t = re.sub(r'[-，+、@,？⊂・ω{}●►❖★！!：\\/(=)…（）『』%《》$;；」:?=<>"／.&#"「_【】]', "",t)    
    # 去除數字
    t = re.sub(r"[0123456789]","",t)    
    # 去除英文  
    t = re.sub(r"[a-zA-Z]","",t)      
    return t  


In [139]:
# 中文斷字，並刪除無意義的符號
import re
import jieba
import jieba.analyse
from optparse import OptionParser

# 設定斷詞字典
jieba.set_dictionary(path+"corpus/dict.txt.big.txt")    
   
# 增加新詞，字典可以隨時叫出修改， 增添新的詞彙。 
jieba.load_userdict(path+ "corpus/userdict.txt")   

seg_list=[]
for t in df.text:
    t= 去特殊符號(t)
    # 結巴分詞，引數為 string。
    result = jieba.cut(t)

    # 讀取中文斷詞結果，存為 string，中間以空白隔開
    seg_string=''
    for word in result:
        seg_string= seg_string + word+' '

    # string 轉為 list
    seg_list.append(seg_string)   


Building prefix dict from d:\My python\corpus\dict.txt.big.txt ...
Loading model from cache C:\Users\terry\AppData\Local\Temp\jieba.ubf87c04fd9101ff99585dd1caa415b1f.cache
Loading model cost 1.004 seconds.
Prefix dict has been built succesfully.


In [146]:
df['token']=seg_list
df.head()

,text,tag,token
0,:怪怪~~這麼牛逼啊，身邊誰再買蒙牛，就罵誰SB！,N,怪怪 這麼 牛 逼 啊 身邊 誰 再 買 蒙牛 就 罵誰
1,“”加錯位置了，抵制蒙牛“很多年”,N,“ ” 加錯 位置 了 抵制 蒙牛 “ 很多年 ”
2,一系列的醜聞表明蒙牛的管理層素質太差，公司治理混亂。消費者應拒絕購買其產品。,N,一系列 的 醜聞 表明 蒙牛 的 管理層 素質 太差 公司 治理 混亂 消費者 應 拒絕 購...
3,一直都在抵制與其抵制日貨，不如抵制蒙牛。,N,一直 都 在 抵制 與其 抵制 日貨 不如 抵制 蒙牛
4,又是蒙牛 再也不能吃蒙牛了，蒙牛的任何產品都不買了,N,又 是 蒙牛 再也不能 吃 蒙牛 了 蒙牛 的 任何 產品 都 不 買 了


In [ ]:
# 加註情感標記
pos_set=[]
neg_set=[]
for d in range(len(df.token)):
    if df.iloc[d]['tag']=="P":
        pos_set.append((df.iloc[d]['token'],'positive'))
    else:
        neg_set.append((df.iloc[d]['token'],'negative'))

In [150]:
pos_set[0:10]  

[('支援 中國 乳業 支援 蒙牛 ', 'positive'),
 ('以後 大家 都 別 買 不 就 行 啦 蒙牛 的 公關 太 厲害 了 得 被 潛 了 多少 回 了 ', 'positive'),
 ('必須 的 全新 蒙牛 幸福 啟航 一起 轉起 來來 了 終於 來 了 ', 'positive'),
 ('回覆 蒙牛 客服 對 意見反饋 及時 值得 肯定 期待 如 你們 所 言 蒙牛 成為 中國 最 安全 乳製品 ', 'positive'),
 ('你好 感謝 對 蒙牛 產品 的 喜愛 呦 我 是 蒙牛 微 客服 歡迎 關注 我 ', 'positive'),
 ('所以 蒙牛 的 奶要 多 給點 信心 要 多 喝 ', 'positive'),
 ('喜歡 喝 蒙牛 不 ', 'positive'),
 ('絕對 是 蒙牛 的 老 酸奶 比如 實 好吃 ', 'positive'),
 ('想 不想 啊 這個 是 現在 我 的 最愛 啊 我 覺得 蒙牛 好 一點 啊 ', 'positive'),
 ('蒙牛 鎮的 很 牛 ', 'positive')]

In [151]:
neg_set[0:10] 

[('怪怪 這麼 牛 逼 啊 身邊 誰 再 買 蒙牛 就 罵誰 ', 'negative'),
 ('“ ” 加錯 位置 了 抵制 蒙牛 “ 很多年 ” ', 'negative'),
 ('一系列 的 醜聞 表明 蒙牛 的 管理層 素質 太差 公司 治理 混亂 消費者 應 拒絕 購買 其 產品 ', 'negative'),
 ('一直 都 在 抵制 與其 抵制 日貨 不如 抵制 蒙牛 ', 'negative'),
 ('又 是 蒙牛 再也不能 吃 蒙牛 了 蒙牛 的 任何 產品 都 不 買 了 ', 'negative'),
 ('已經 不 喝牛奶 了 已經 不買 蒙牛 了 次 奧次 奧 還有 人買 蒙牛 ', 'negative'),
 ('已經 不買 蒙牛 好幾個 月 了 堅持 到 丫 倒閉 ', 'negative'),
 ('不 愛喝 蒙牛 以後 也 不會 喝 ', 'negative'),
 ('牛奶 永不 喝 蒙牛 蒙牛 乳業 ', 'negative'),
 ('牛 逼 蒙牛 牛 逼 ', 'negative')]

## 開始訓練

In [152]:
# ngrams 函數，以多個字元為分析單位
def ngrams(input, min, max):
    input = input.split(' ')
    output = []
    texts=''  
    for c in range(min,max+1):
        n=c
        for i in range(len(input)-n+1):
            texts=' '.join(input[i:i+n])
            output.append(texts)
    return output

## train_set, 訓練資料

In [169]:
train_set = []
# 把測試組的所有資料叫進來，含字詞及情緒標記 #######
for (words, sentiment) in pos_set + neg_set:
    # 所有的字詞 與 情緒標記
    word_list=ngrams(words, 1,1) 
    
    # 去除停用字 stopwords
    stopwords=['我', '很', '他', '的', '也' ,"內容"]   
    word_list=[e for e  in word_list if e not in stopwords] 
   
    # 字長控制
    words_filtered = [e for e in word_list if len(e) >= 1]        
    train_set.append((words_filtered, sentiment))

In [170]:
import nltk
# 取得文本中所有的字詞 list，有重覆
def get_words_in_train(train_set):
    all_words = []
    for (words, sentiment) in train_set:
        all_words.extend(words)
    return all_words
# 取得字詞列表，不重復 dictionary_key
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features
# 字詞是否(True or False)在文件有被引用，dictionary
def extract_features(train_set):
    document_words = set(train_set)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

## 分類器，要花時間

In [171]:
# 字詞列表， dictionary_key 不重復
word_features = get_word_features(get_words_in_train(train_set))

# 文字袋比對，每一筆資料，字詞的有無，注意引數 extract_features, tweets　
training = nltk.classify.apply_features(extract_features, train_set)

# 這要花時間 ############
# 找出潛在的規則
classifier = nltk.NaiveBayesClassifier.train(training)

In [172]:
# 列出最重要的 50 條規則
print (classifier.show_most_informative_features(50))

Most Informative Features
            contains(時尚) = True           positi : negati =     42.1 : 1.0
            contains(宕機) = True           negati : positi =     34.3 : 1.0
            contains(鬱悶) = True           negati : positi =     28.8 : 1.0
            contains(小巧) = True           positi : negati =     28.8 : 1.0
            contains(抵制) = True           negati : positi =     24.6 : 1.0
            contains(根本) = True           negati : positi =     24.0 : 1.0
            contains(漂亮) = True           positi : negati =     22.9 : 1.0
            contains(垃圾) = True           negati : positi =     21.7 : 1.0
            contains(夠用) = True           positi : negati =     21.6 : 1.0
            contains(很差) = True           negati : positi =     20.9 : 1.0
            contains(正品) = True           positi : negati =     20.9 : 1.0
            contains(遠離) = True           negati : positi =     20.3 : 1.0
            contains(感謝) = True           positi : negati =     20.1 : 1.0

## save classifier 

In [173]:
import pickle
save_classifier = open(path+"data/naivebayes_senti.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [174]:
# load classifier
import pickle
classifier_f = open(path+"data/naivebayes_senti.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

## test_set 測試資料

In [195]:
test_set =[('作者列舉的方法太麻煩，配料也不好找。不是太實用。','negative'),
           ('那房子令人討厭','negative'),
           ('作者有一種專業的謹慎，全書結構簡單，但内容詳實。', 'positive'),
           ('最大的缺陷是容易養成中文式英語','negative'),
           ('那首歌太吵了','negative'),           
           ('早上我覺得心情很好', 'positive'),
           ('昨天晚上的音樂會令人感動', 'positive')
           ]    

In [196]:
# 中文斷字，並刪除無意義的符號
import re
import jieba
import jieba.analyse
from optparse import OptionParser

# 設定斷詞字典
jieba.set_dictionary(path+"corpus/dict.txt.big.txt")    
   
# 增加新詞，字典可以隨時叫出修改， 增添新的詞彙。 
jieba.load_userdict(path+ "corpus/userdict.txt")   

test_token=[]
for n in range(len(test_set)):
    result = jieba.cut(test_set[n][0])
    # 結巴分詞，引數為 string。
    texts=''
    for t in result:
       texts=texts+ ' '+ t
    test_token.append(texts)

Building prefix dict from d:\My python\corpus\dict.txt.big.txt ...
Loading model from cache C:\Users\terry\AppData\Local\Temp\jieba.ubf87c04fd9101ff99585dd1caa415b1f.cache
Loading model cost 1.006 seconds.
Prefix dict has been built succesfully.


In [197]:
test_token

[' 作者 列舉 的 方法 太 麻煩 ， 配料 也 不好 找 。 不是 太 實用 。',
 ' 那 房子 令人討厭',
 ' 作者 有 一種 專業 的 謹慎 ， 全書 結構 簡單 ， 但 内容 詳實 。',
 ' 最大 的 缺陷 是 容易 養成 中文 式 英語',
 ' 那首歌 太吵 了',
 ' 早上 我 覺得 心情 很 好',
 ' 昨天晚上 的 音樂會 令人感動']

In [198]:
# 正確率計算
yes=0
for t in range(len(test_token)):
    pred= classifier.classify(extract_features(test_token[t].split()))
    n+=1
    print (pred, test_set[t][1])    
    
    if pred == test_set[t][1]:
        yes+=1
acc=yes/len(test_set)
print (acc)

negative negative
positive negative
negative positive
negative negative
negative negative
positive positive
positive positive
0.7142857142857143


## target 標的資料

In [199]:
target =['他 確實 是 普普通通 啦',
         '我 真 的 不 知道',  
         '沒有 什麼 特別 的 感覺']

for t in target:
    print (classifier.classify(extract_features(t.split())))

positive
negative
negative


# 作業
* 建置自己的訓練組，正向語料與負向語料。正負向語料各一百則。
* 語料為任選一段文字，加情緒註記，'negative','positive'。
* 存成 pickle 檔，檔名為 pos_set.pickle, neg_set.pickl。交作業時一起交。
* 製作分類器。
* 測試正確度。
